<a href="https://colab.research.google.com/github/surensnyper/class_machine_vision/blob/main/CameraCalibration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
n# https://www.youtube.com/watch?v=E3Lg4aZVCAU - Tutorial video of OpenCV
# https://www.youtube.com/watch?v=3h7wgR5fYik&t=760s - Another Tutorial
# https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html - OpenCV
import numpy as np
import cv2 as cv
import glob

In [ ]:
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt

# img = "./5.jpg"
# img2 = cv.imread(img)
# cv2_imshow(img2)

In [ ]:
# chessboardSize = (9, 7)
# frameSize = (1440, 1080)

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [ ]:
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1,2)
# objp = np.zeros((chessboardSize[0]*chessboardSize[1],3), np.float32)
# objp[:,:2] = np.mgrid[0:chessboardSize[0], 0:chessboardSize[1]].T.reshape(-1,2)

In [ ]:
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
images = glob.glob('*.jpg')

In [ ]:
for fname in images:

    # print(fname)
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,6), None)
    # ret, corners = cv.findChessboardCorners(gray, chessboardSize, None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,6), corners2, ret)
#       cv.imshow('img', img)
#       cv.waitKey(500)
# cv.destroyAllWindows()

In [ ]:
################## CALIBRATION ###################
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, frameSize, None, None)

print("Camera Calibrated: ", ret)
print("\nCamera Matrix: \n", mtx)
print("\nDistortion Parameters: \n", dist)
print("\nRotation Vectors: \n", rvecs)
print("\nTranslation Vectors: \n", tvecs)

Camera Calibrated:  0.7792198996322665

Camera Matrix: 
 [[919.07664386   0.         480.21396913]
 [  0.         924.80151125 646.07406349]
 [  0.           0.           1.        ]]

Distortion Parameters: 
 [[ 2.43439483e-03  5.03258218e-01  6.31106186e-03 -1.08108075e-03
  -1.31973977e+00]]

Rotation Vectors: 
 [array([[-0.08408733],
       [-0.09864599],
       [ 3.13811342]]), array([[-0.52831046],
       [ 0.4514259 ],
       [ 3.05966441]]), array([[-0.07833323],
       [ 0.14497768],
       [-3.13038688]]), array([[ 0.02074186],
       [-0.47697786],
       [ 3.08039953]]), array([[-0.08319506],
       [ 0.1480352 ],
       [-3.13087952]]), array([[-0.16566147],
       [ 0.24940791],
       [-3.10841213]])]

Translation Vectors: 
 [array([[ 3.75523795],
       [ 0.96536826],
       [11.23573594]]), array([[ 3.97734711],
       [ 0.91119825],
       [11.6531275 ]]), array([[ 3.11339442],
       [ 1.23319237],
       [10.43669757]]), array([[ 3.48849384],
       [ 2.35492371],
 

In [ ]:
################## UNDISTORTION ###################
img = cv.imread('12.jpg')
h,  w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)
# crop the image
x, y, w, h = roi
dst = dst[y:y+h, x:x+w]
cv.imwrite('calibresult1.png', dst)

True

In [ ]:
# img3 = "./calibresult.png"
# img4 = cv.imread(img3)
# cv2_imshow(img4)

In [ ]:
# Reprojection Error

mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv.norm(imgpoints[i], imgpoints2, cv.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )

total error: 0.11684156503410041
